Задача 1. 

In [50]:
def kth_root_test(x, a, k):
    return x ** k - a
     
def bisection(g, left_edge, right_edge, num_iters=10000):
    assert(g(left_edge) * g(right_edge) < 0)
    left = left_edge
    right = right_edge
    for _ in range(num_iters):
        mid = (left + right) / 2
        if g(left) * g(mid) < 0:
            right = mid
        else:
            left = mid   
    return (left + right) / 2

bisection(lambda x: kth_root_test(x, -99, 5), -1000000, 10000000)

-2.5068424421341

In [51]:
def derivative(k, x):
    return k * (x **(k - 1))

def Newton(g, dg, cur, num_iters=1000):
    for _ in range(num_iters):
        cur = cur - g(cur)/dg(cur) 
    return cur    

Newton(lambda x: kth_root_test(x, -99, 5), lambda x: derivative(5, x), 10000000)   

-2.5068424421341002

Задача 2. 

In [53]:
import numpy as np 

def fragmentation(f, l, r):
    sectors = np.linspace(l, r, 100985)
    res = []
    for i, x in enumerate(sectors[:-1]):
        if (f(x) * f(sectors[i + 1])) < 0:
            res.append((x, sectors[i + 1]))
    return res        
    

In [54]:
poly = lambda coefs, x: sum(coefs[i] * x** i for i in range(len(coefs)))
dpoly =  lambda coefs, x: sum(coefs[i + 1] * (i + 1) * x** i for i in range(len(coefs) - 1))

cf = [0, -2, 1]

frag = fragmentation(lambda x: poly(cf, x), -10, 20)
print(frag)


[(-9.902558821295315e-05, 0.00019805117642412995), (1.9998217539412178, 2.000118830705855)]


In [55]:
def roots(f, l, r) :
    return [bisection(f, l, r) for l,r in fragmentation(f, l, r)] 

print(roots(lambda x: poly(cf, x), -10, 20))

[1.3160731531800647e-162, 2.000000000000001]


In [56]:
def minimum(coefs, l , r):
    candidates = [r]
    candidates += roots(lambda x: dpoly(coefs, x), l, r)
    mini = poly(coefs, l)
    res = l
    for x in candidates:
        if poly(coefs, x) < mini:
            res = x
            mini = poly(coefs, x)
        
    return mini, res

minimum(cf, -10, 20)

(-1.0, 1.0000000000000004)

Задача 3. 

Наша функция выпукла как линейная комбинация выпуклых с положительными коэффициентами.
Обратим внимание на производную: $ae^x - be^{-bx} + 2c(x - d)$. Поскольку она монотонно возрастает с ростом $x$, то исходная функция имеет один экстремум, причем это именно минимум. Получается, достаточно вычислить корень производной. Этим мы и займемся.

In [57]:
def funcs(a, b, c, d):
    f = lambda x: np.exp(a * x) + np.exp(- b * x) + c * (x - d) ** 2 
    df = lambda x: a * np.exp(a * x) - b * np.exp(- b * x) + 2 * c * (x - d)
    d2f = lambda x: a ** 2 * np.exp(a * x) + b ** 2 * np.exp(- b * x) + 2 * c
    return f, df, d2f

In [58]:
def tern(f, left, right, eps = 1e-9):
    while (right - left) > eps:
        l = (2 * left + right) / 3
        r = (2 * right + left) / 3
        if f(l) > f(r):
            left = l
        else:
            right = r   
    return (left + right) / 2        

In [59]:
tern(lambda x: poly([0, -2, 1], x), 0, 2)

0.9999999928171579

In [60]:
f, df, d2f = funcs(10, 12, 15, 13)
b_ans = bisection(df, -30, 30)
n_ans = Newton(df, d2f, 0)
t_ans = tern(f, -30, 30)

print(b_ans, f(b_ans))
print(n_ans, f(n_ans))
print(t_ans, f(t_ans))


0.3635600302816955 2433.131582614239
0.3635600302816955 2433.131582614239
0.3635600178201891 2433.131582614239
